# Fast SLAM 2.0

Fast SLAM 2.0 with multiple measurements.

Fast SLAM 1.0 samples poses based on the control $u_t$ only, and then uses the measurement $z_t$ to calculate importance weights.

Fast SLAM 2.0 samples poses based on the measurement $z_t$ in addition to the control $u_t$

### Pose Update $x^{[k]}_t$

Algorithm Fast SLAM 2.0 stated in the book estimate robot poses $x^{[k]}_{t,j}$ for each map feature $j$.

Pose for map feature $j$
 - Mean: $\mu_{x_t,j}$
 - Covariance: $\Sigma_{x,j}$

Let's define the optimal pose distribution as

$p(x_t) = \Pi_j p(x_{t,j} \mid z_t, u_t)$

$p(x_t) = \eta \exp \left\lbrace -J_t \right\rbrace$

$J_t = \sum_j \frac{1}{2}\left(x_t - \mu_j\right)^T \Sigma^{-1}_{x,j}\left(x_t - \mu_j\right)$

Mean

$\frac{\partial J_t}{\partial x_t} = \sum_j \Sigma^{-1}_{x,j}\left(\mu_t - \mu_j\right) = 0$

$\sum_j \Sigma^{-1}_{x,j} \mu_t = \sum_j \Sigma^{-1}_{x,j}\mu_j $

$\mu_t = \left(\sum_j \Sigma^{-1}_{x,j}\right)^{-1}\sum_j \Sigma^{-1}_{x,j}\mu_j $

$= \left(\Sigma^{-1}_{x,0} + \Sigma^{-1}_{x,1} + \cdots  + \Sigma^{-1}_{x,N-1}\right)^{-1}\left(\Sigma^{-1}_{x,0}\mu_{0} + \Sigma^{-1}_{x,1}\mu_{1} + \cdots  + \Sigma^{-1}_{x,N-1}\mu_{N-1} \right)$

Covariance

$\frac{\partial^2 J_t}{\partial x_t^2} = \sum_j \Sigma^{-1}_{x,j}$

$\Sigma_x = \left(\sum_j \Sigma^{-1}_{x,j}\right)^{-1}$

$= \left(\Sigma^{-1}_{x,0} + \Sigma^{-1}_{x,1} + \cdots  + \Sigma^{-1}_{x,N-1}\right)^{-1} $



### Algorithm FastSLAM 2.0 ($z_t$, $u_t$, $Y_{t-1}$)

for $k = 0$ to $M-1$ do
- Retrieve a particle from $Y_{t-1}$
- $\hat{x}_{j,t} = g\left(x^{[k]}_{t-1}, u_t\right)$ Predict Pose

- for each feature $j=0$ to $N^{[k]}_{t-1}-1$
  - $\bar{z}_j = h\left(\mu^{[k]}_{j,t-1}, \hat{x}_{t}\right)$ Predict measurement 1
  - $H_{x,j} = \nabla_{x_t} h\left(\mu^{[k]}_{j,t-1}, \hat{x}_{j,t} \right)$ Jacobian with respect to pose
  - $H_{m,j} = \nabla_{m_j} h\left(\mu^{[k]}_{j,t-1}, \hat{x}_{j,t} \right)$ Jacobian with respect to map feature
  - $Q_j = Q_t + H_{m,j} \Sigma^{[k]}_{j,t-1} H^T_{m,j}$ Measurement information
  - $\Sigma_{x,j} = \left[H^T_{x,j} Q^{-1}_{j} H_{x,j} + R^{-1}\right]^{-1}$ Covariance of proposal distribution

- for each measurement $z_i$
  - for each feature $j=0$ to $N^{[k]}_{t-1}-1$
    - $\pi_j = \left|2\pi Q_j\right|^{-\frac{1}{2}} exp\left\lbrace-\frac{1}{2} \left(z_t-\bar{z}_j\right)^T Q^{-1}_j \left(z_t-\bar{z}_j\right) \right\rbrace$ Correspondence likelihood
    - find the optimal correspondence

  - if the optimum correspondence for $z_i$ is not found
    - $x^{[k]}_{t,j} \sim p\left(x_t \mid x^{[k]}_{t-1}, u_t\right)$ sample pose
    - $\mu^{[k]}_{j,t} = h^{-1}\left(z_t, x^{[k]}_t\right)$ initialize mean
    - $H_{m,j} = \nabla_{m_j} h\left(\mu^{[k]}_{j,t}, x^{[k]}_t\right)$ Jacobian with respect to map feature
    - $\Sigma^{[k]}_{j,t} = \left(H^{-1}_{m,j}\right)^T Q_t H^{-1}_{m,j}$ initialize covariance
    - $i^{[k]}_{j,t} = 1$ initialize counter
    - $w^{[k]} = p_0$ importance weight
  - else if the optimum correspondence is found
    - $\mu_{x_t, j} = \hat{x}_{j,t} + \Sigma_{x,j} H^T_{x,j} Q^{-1}_j \left(z_t - \bar{z}_j\right)$ mean of proposal distribution      

  - compute summations
    - $\sum_j \Sigma^{-1}_{x,j}$
    - $\sum_j \Sigma^{-1}_{x,j}\mu_j$

- compute the mean and variance of the pose
- $\mu_t = \left(\sum_j \Sigma^{-1}_{x,j}\right)^{-1}\sum_j \Sigma^{-1}_{x,j}\mu_j $
- $\Sigma_x = \left(\sum_j \Sigma^{-1}_{x,j}\right)^{-1}$
- $x^{[k]}_{t} \sim \mathcal{N}\left(\mu_{x_{t}}, \Sigma_{x}\right)$  sample pose

- for each feature $j=0$ to $N^{[k]}_{t}-1$
  - if the correspondence is found
    - $K = \Sigma^{[k]}_{j,t-1} H^T_{m,j}Q^{-1}_j$ calculate Kalmangain
    - $\hat{z}_j = h\left(\mu^{[k]}_{j,t-1}, x^{[k]}_{t}\right)$ Predict measurement 2
    - $\mu^{[k]}_{j,t} = \mu^{[k]}_{j,t-1} + K\left(z_t - \hat{z}_j\right)$ update mean
    - $\Sigma^{[k]}_{j,t} = \left(I - K H_{m,j}\right) \Sigma^{[k]}_{j,t-1}$ update covariance
    - $i^{[k]}_{j,t} = i^{[k]}_{j,t-1} + 1$ increment counter
    - $L = H_{x,j}R_tH^T_{x,j} + H_{m,j}\Sigma^{[k]}_{j,t-1}H^T_{m,j} + Q_t$
    - $w^{[k]}_j = \left|2\pi L\right|^{-\frac{1}{2}} \exp\left\lbrace -\frac{1}{2}\left(z_t - \hat{z}_j \right)^TL^{-1}\left(z_t - \hat{z}_j \right)\right\rbrace$ importance weight

